In [ ]:
# # list all the prompts
# for phase_dialogue, cicero_power, human_power in get_all_phase_dialogues():
#     print(f"Cicero is: {cicero_power}\n")
#     text = build_dialogue_text(phase_dialogue)
#     print(text)
#     print("=====================================================\n\n")

In [ ]:
# # answer random dialogue
# dialogue, cicero_power, human_power = random.choice(list(get_all_phase_dialogues()))

# text = build_dialogue_text(dialogue)
# prompt = cooperation_prompt1.format(cicero_power=cicero_power, human_power=human_power)

# print(f"( Cicero is: {cicero_power} )\n")
# print(text + prompt)

# response = openai.Completion.create(
#     model="text-davinci-003",
#     # model="text-curie-001",
#     prompt=text + prompt,
#     max_tokens=300,
# )["choices"][0]["text"]

# print(response)